In [1]:
import numpy as np

from pyquil import Program, get_qc
from pyquil.gates import CZ, H, I, X, MEASURE
from pyquil.quilbase import Declare
from scipy.linalg import expm
from scipy.stats import binom


def damping_channel(damp_prob=.1):
    """
    Generate the Kraus operators corresponding to an amplitude damping
    noise channel.

    :params float damp_prob: The one-step damping probability.
    :return: A list [k1, k2] of the Kraus operators that parametrize the map.
    :rtype: list
    """
    damping_op = np.sqrt(damp_prob) * np.array([[0, 1],
                                                [0, 0]])

    residual_kraus = np.diag([1, np.sqrt(1-damp_prob)])
    #print(residual_kraus, damping_op)
    return [residual_kraus, damping_op]

def append_kraus_to_gate(kraus_ops, g):
    """
    Follow a gate `g` by a Kraus map described by `kraus_ops`.

    :param list kraus_ops: The Kraus operators.
    :param numpy.ndarray g: The unitary gate.
    :return: A list of transformed Kraus operators.
    """
    return [kj.dot(g) for kj in kraus_ops]


def append_damping_to_gate(gate, damp_prob=.1):
    """
    Generate the Kraus operators corresponding to a given unitary
    single qubit gate followed by an amplitude damping noise channel.

    :params np.ndarray|list gate: The 2x2 unitary gate matrix.
    :params float damp_prob: The one-step damping probability.
    :return: A list [k1, k2] of the Kraus operators that parametrize the map.
    :rtype: list
    """
    return append_kraus_to_gate(damping_channel(damp_prob), gate)

def take_counts(array):
    dic = dict()
    for a in array:
        if tuple(k for k in a) in dic:
            dic[tuple(k for k in a)] += 1
        else:
            dic[tuple(k for k in a)] = 1
    return dic

In [18]:
from pyquil.api import local_forest_runtime
with local_forest_runtime():
    
    qc = get_qc('2q-qvm')
    # single step damping probability
    damping_per_I = 0.02

    # number of program executions
    trials = 200

    results_damping = []
    lengths = np.arange(0, 201, 10, dtype=int)
    for jj, num_I in enumerate(lengths):
        p = Program(
            Declare("ro", "BIT", 1),
            X(0),
        )
        # want increasing number of I-gates
        p.inst([I(0) for _ in range(num_I)])
        p.inst(MEASURE(0, ("ro", 0)))

        # overload identity I on qc 0
        p.define_noisy_gate("I", [0], append_damping_to_gate(np.eye(2), damping_per_I))
        #print(append_damping_to_gate(np.eye(2), damping_per_I))
        p.wrap_in_numshots_loop(trials)
        qc.qam.random_seed = int(num_I)
        res = qc.run(p).get_register_map().get("ro")
        print(take_counts(res))
        results_damping.append([np.mean(res), np.std(res) / np.sqrt(trials)])

    results_damping = np.array(results_damping)


TypeError: 'numpy.ndarray' object is not callable

In [ ]:

dense_lengths = np.arange(0, lengths.max()+1, .2)
survival_probs = (1-damping_per_I)**dense_lengths
logpmf = binom.logpmf(np.arange(trials+1)[np.newaxis, :], trials, survival_probs[:, np.newaxis])/np.log(10)

DARK_TEAL = '#48737F'
FUSCHIA = "#D6619E"
BEIGE = '#EAE8C6'

import matplotlib.colors as colors
import matplotlib.pyplot as plt
cm = colors.LinearSegmentedColormap.from_list('anglemap', ["white", FUSCHIA, BEIGE], N=256, gamma=1.5)

plt.figure(figsize=(14, 6))
plt.pcolor(dense_lengths, np.arange(trials+1)/trials, logpmf.T, cmap=cm, vmin=-4, vmax=logpmf.max())
plt.plot(dense_lengths, survival_probs, c=BEIGE, label="Expected mean")
plt.errorbar(lengths, results_damping[:,0], yerr=2*results_damping[:,1], c=DARK_TEAL,
            label=r"noisy qvm, errorbars $ = \pm 2\hat{\sigma}$", marker="o")
cb = plt.colorbar()
cb.set_label(r"$\log_{10} \mathrm{Pr}(n_1; n_{\rm trials}, p_{\rm survival}(t))$", size=20)

plt.title("Amplitude damping model of a single qubit", size=20)
plt.xlabel(r"Time $t$ [arb. units]", size=14)
plt.ylabel(r"$n_1/n_{\rm trials}$", size=14)
plt.legend(loc="best", fontsize=18)
plt.xlim(*lengths[[0, -1]])
plt.ylim(0, 1)

plt.savefig("test.png")

In [65]:
import numpy as np
# Define Pauli operations using PauliTerm
# Defining the Pauli matrices
I = np.array([[1.+0.j, 0.+0.j],
            [0.+0.j, 1.+0.j]])

X = np.array([[0.+0.j, 1.+0.j],
            [1.+0.j, 0.+0.j]])

Y = np.array([[0.+1.j, 0.+0.j],
            [0.+0.j, 0.-1.j]])

Z = np.array([[1.+0.j, 0.+0.j],
            [0.+0.j, -1.+0.j]])

cnot = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 0, 1],
                [0, 0, 1, 0]])

# Pauli products
pauli_YZ = np.kron(Y, Z)
pauli_IY = np.kron(I, Y)
pauli_YY = np.kron(Y, Y)
pauli_XY = np.kron(X, Y)
pauli_IX = np.kron(I, X)
pauli_II = np.kron(I, I)

print(np.matmul(cnot,pauli_II))

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]]


In [54]:
from TrotterExample import get_noise_model, executor, get_backend
from pyquil import Program
from pyquil.gates import H, CNOT, Z, MEASURE, S, X, Y, I, RX, RZ, FENCE
from pyquil.quilbase import Declare
from pyquil.api import local_forest_runtime
import numpy as np

with local_forest_runtime():

    # Here the backend for the simulation is prepared
    from pyquil.quantum_processor import NxQuantumProcessor
    from pyquil.noise import NoiseModel
    from pyquil import get_qc
    import networkx as nx
    backend = get_qc("5q-qvm") #str(n)+'q-qvm' #args.backend
    isa = backend.to_compiler_isa()
    backend_qubits = sorted(int(k) for k in isa.qubits.keys())
    # By default every qubit on our fake backend is connected with every other qubit. This breaks the algorhythm. So we need to change the topology
    # So this line here changes the topology of the fake backend. The condition in the end determines it
    edges = [(q1, q2) for q1 in backend_qubits for q2 in backend_qubits if abs(q1-q2) == 1]
    # Build the NX graph
    topo = nx.from_edgelist(edges)
    # You would uncomment the next line if you have disconnected qubits
    # topo.add_nodes_from(qubits)
    nx_quantum_processor = NxQuantumProcessor(topo)
    backend.compiler.quantum_processor = nx_quantum_processor

    prog = Program()
    prog += Declare("ro", "BIT", 2)
    #prog += X(0)
    prog += X(1)
    prog += CNOT(0,1)
    prog += MEASURE(0, ("ro", 0))
    prog += MEASURE(1, ("ro", 1))
    identity = np.eye(2)
    print(identity)
    identity2 = np.eye(4)
    print(identity2)
    flip =  np.array([[0.+0.j, 1.+0.j],
                            [1.+0.j, 0.+0.j]])
    print(flip)
    test = [np.array([[1.        , 0.        ],
       [0.        , 0.98994949]]), np.array([[0.        , 0.14142136],
       [0.        , 0.        ]])]
    (noise_model, twoqubit_error_template, singlequbit_error_template) = get_noise_model()
    #prog.define_noisy_gate("X", [0], [identity])
    prog.define_noisy_gate("CNOT", [0,1], [cnot])#append_damping_to_gate(np.eye(2), 0.1))
    #prog.define_noisy_gate("CNOT", [1,0], [pauli_IX])

    counts = executor([prog], backend, 1024)
    print(counts)
    #print(noise_model)
    

[[1. 0.]
 [0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[{(0, 1): 1024}]
